# Apply FT

In [1]:
#%cd ./flosp
import pandas as pd
import numpy as np
import seaborn as sns
import featuretools as ft
#import flosp
#import bradlib as bl
sns.set()

from core import create_timeseries_from_events, make_callender_columns

# get data

In [2]:
data = ft.demo.load_mock_customer()

In [3]:
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])

In [4]:
transactions_df['arrive_dt'] = transactions_df['transaction_time']
transactions_df['depart_dt'] = transactions_df['transaction_time'] + pd.Timedelta(2,'h')

In [5]:
transactions_df['date_hour'] = transactions_df.arrive_dt.apply(lambda x : x.replace(second=0,minute=0) )

#### make es

In [8]:
transactions_df.head(2)

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code,join_date,arrive_dt,depart_dt,date_hour
0,352,1,2014-01-01 00:00:00,4,7.39,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 00:00:00,2014-01-01 02:00:00,2014-01-01
1,186,1,2014-01-01 00:01:05,4,147.23,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 00:01:05,2014-01-01 02:01:05,2014-01-01


In [6]:
es = ft.EntitySet(id='activity')

In [19]:
es = es.entity_from_dataframe(entity_id='transactions',
                        dataframe=transactions_df,
                        index='transaction_id',
                        time_index='transaction_time',
                              secondary_time_index={'depart_dt':['amount']}
                             )

In [20]:
es['transactions'].df.head(2)

,transaction_id,session_id,transaction_time,amount,customer_id,device,session_start,zip_code,join_date,arrive_dt,depart_dt,date_hour,product_id
352,352,1,2014-01-01 00:00:00,7.39,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 00:00:00,2014-01-01 02:00:00,2014-01-01,4
186,186,1,2014-01-01 00:01:05,147.23,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 00:01:05,2014-01-01 02:01:05,2014-01-01,4


In [21]:
es['transactions'].secondary_time_index

{'depart_dt': ['amount', 'depart_dt']}

In [25]:
es['transactions'].last_time_index.head()

352   2014-01-01 00:00:00
186   2014-01-01 00:01:05
319   2014-01-01 00:02:10
256   2014-01-01 00:03:15
449   2014-01-01 00:04:20
Name: transaction_time, dtype: datetime64[ns]

In [23]:
es.add_last_time_indexes()

es = es.normalize_entity(base_entity_id='transactions',
                   new_entity_id="hours",
                    index="date_hour")

es['hours'].df

In [15]:

def absolute(column1,column2):
    res = abs(column1) + abs(column2)
    return res
 

Absolute = make_trans_primitive(function=absolute,
                                 input_types=[Numeric,Numeric],
                                 return_type=Numeric)

fm, feature_defs = ft.dfs(entityset=es,target_entity='transactions',
      agg_primitives=['count'],
      trans_primitives=[Absolute],
    max_depth=1,n_jobs=1)

fm.head(2)

In [25]:
es['hours'].df

,date_hour,first_transactions_time
2014-01-01 00:00:00,2014-01-01 00:00:00,2014-01-01 00:00:00
2014-01-01 01:00:00,2014-01-01 01:00:00,2014-01-01 01:00:40
2014-01-01 02:00:00,2014-01-01 02:00:00,2014-01-01 02:00:15
2014-01-01 03:00:00,2014-01-01 03:00:00,2014-01-01 03:00:55
2014-01-01 04:00:00,2014-01-01 04:00:00,2014-01-01 04:00:30
2014-01-01 05:00:00,2014-01-01 05:00:00,2014-01-01 05:00:05
2014-01-01 06:00:00,2014-01-01 06:00:00,2014-01-01 06:00:45
2014-01-01 07:00:00,2014-01-01 07:00:00,2014-01-01 07:00:20
2014-01-01 08:00:00,2014-01-01 08:00:00,2014-01-01 08:01:00
2014-01-01 09:00:00,2014-01-01 09:00:00,2014-01-01 09:00:35


In [38]:
from featuretools.primitives import make_agg_primitive, make_trans_primitive

from featuretools.variable_types import Text, Numeric, Datetime

def occupancy(column1,column2,time):
    print(time)
    return(4)

Occupancy = make_agg_primitive(function=occupancy,
                           input_types=[Datetime,Datetime],
                            return_type=Numeric,
                            uses_calc_time=True)

In [39]:
feature_matrix1, feature_defs = ft.dfs(entityset=es,target_entity='hours',
      agg_primitives=[Occupancy],
      trans_primitives=['month'],
    max_depth=2,n_jobs=1)

feature_matrix1.head()

2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2018-12-04 09:33:41.848473
2

,"OCCUPANCY(transactions.transaction_time, depart_dt)","OCCUPANCY(transactions.session_start, join_date)","OCCUPANCY(transactions.transaction_time, session_start)","OCCUPANCY(transactions.transaction_time, join_date)","OCCUPANCY(transactions.join_date, depart_dt)","OCCUPANCY(transactions.depart_dt, transaction_time)","OCCUPANCY(transactions.join_date, transaction_time)","OCCUPANCY(transactions.session_start, transaction_time)","OCCUPANCY(transactions.join_date, session_start)","OCCUPANCY(transactions.depart_dt, session_start)","OCCUPANCY(transactions.session_start, depart_dt)","OCCUPANCY(transactions.depart_dt, join_date)",MONTH(first_transactions_time)
date_hour,,,,,,,,,,,,,
2014-01-01 00:00:00,4,4,4,4,4,4,4,4,4,4,4,4,1
2014-01-01 01:00:00,4,4,4,4,4,4,4,4,4,4,4,4,1
2014-01-01 02:00:00,4,4,4,4,4,4,4,4,4,4,4,4,1
2014-01-01 03:00:00,4,4,4,4,4,4,4,4,4,4,4,4,1
2014-01-01 04:00:00,4,4,4,4,4,4,4,4,4,4,4,4,1


In [29]:
es['transactions'].df.head()

,arrive_dt,transaction_id,session_id,transaction_time,amount,customer_id,device,session_start,zip_code,join_date,depart_dt,date_hour,product_id
2014-01-01 00:00:00,2014-01-01 00:00:00,352,1,2014-01-01 00:00:00,7.39,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:00:00,2014-01-01,4
2014-01-01 00:01:05,2014-01-01 00:01:05,186,1,2014-01-01 00:01:05,147.23,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:01:05,2014-01-01,4
2014-01-01 00:02:10,2014-01-01 00:02:10,319,1,2014-01-01 00:02:10,111.34,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:02:10,2014-01-01,2
2014-01-01 00:03:15,2014-01-01 00:03:15,256,1,2014-01-01 00:03:15,78.15,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:03:15,2014-01-01,4
2014-01-01 00:04:20,2014-01-01 00:04:20,449,1,2014-01-01 00:04:20,33.93,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:04:20,2014-01-01,3


In [99]:
es['hours'].df

,date_hour,first_transactions_time
2014-01-01 00:00:00,2014-01-01 00:00:00,2014-01-01 00:00:00
2014-01-01 01:00:00,2014-01-01 01:00:00,2014-01-01 01:00:40
2014-01-01 02:00:00,2014-01-01 02:00:00,2014-01-01 02:00:15
2014-01-01 03:00:00,2014-01-01 03:00:00,2014-01-01 03:00:55
2014-01-01 04:00:00,2014-01-01 04:00:00,2014-01-01 04:00:30
2014-01-01 05:00:00,2014-01-01 05:00:00,2014-01-01 05:00:05
2014-01-01 06:00:00,2014-01-01 06:00:00,2014-01-01 06:00:45
2014-01-01 07:00:00,2014-01-01 07:00:00,2014-01-01 07:00:20
2014-01-01 08:00:00,2014-01-01 08:00:00,2014-01-01 08:01:00
2014-01-01 09:00:00,2014-01-01 09:00:00,2014-01-01 09:00:35


In [127]:
es['transactions'].df.head(2)

,arrive_dt,transaction_id,session_id,transaction_time,amount,customer_id,device,session_start,zip_code,join_date,depart_dt,date_hour,product_id
2014-01-01 00:00:00,2014-01-01 00:00:00,352,1,2014-01-01 00:00:00,7.39,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:00:00,2014-01-01,4
2014-01-01 00:01:05,2014-01-01 00:01:05,186,1,2014-01-01 00:01:05,147.23,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:01:05,2014-01-01,4


In [148]:
df_ct = pd.DataFrame(es['hours'].df['date_hour'].copy()).reset_index().rename(columns={'date_hour':'time','index':'date_hour'})
df_ct['time'] = df_ct['time'].apply(lambda x : x - pd.Timedelta(1,'h'))
df_ct

,date_hour,time
0,2014-01-01 00:00:00,2013-12-31 23:00:00
1,2014-01-01 01:00:00,2014-01-01 00:00:00
2,2014-01-01 02:00:00,2014-01-01 01:00:00
3,2014-01-01 03:00:00,2014-01-01 02:00:00
4,2014-01-01 04:00:00,2014-01-01 03:00:00
5,2014-01-01 05:00:00,2014-01-01 04:00:00
6,2014-01-01 06:00:00,2014-01-01 05:00:00
7,2014-01-01 07:00:00,2014-01-01 06:00:00
8,2014-01-01 08:00:00,2014-01-01 07:00:00
9,2014-01-01 09:00:00,2014-01-01 08:00:00


In [149]:
es

Entityset: activity
  Entities:
    transactions [Rows: 500, Columns: 13]
    hours [Rows: 10, Columns: 2]
  Relationships:
    transactions.date_hour -> hours.date_hour

In [151]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                     target_entity="hours",
                                    agg_primitives=['count'],
                                    trans_primitives=[],
                                cutoff_time =df_ct,
                                     max_depth=1)
feature_matrix

,COUNT(transactions)
date_hour,
2014-01-01 00:00:00,0
2014-01-01 01:00:00,0
2014-01-01 02:00:00,0
2014-01-01 03:00:00,0
2014-01-01 04:00:00,0
2014-01-01 05:00:00,0
2014-01-01 06:00:00,0
2014-01-01 07:00:00,0
2014-01-01 08:00:00,0


In [122]:
ft.primitives.list_primitives()

,name,type,description
0,mode,aggregation,Finds the most common element in a categorical...
1,last,aggregation,Returns the last value.
2,median,aggregation,Finds the median value of any feature with wel...
3,min,aggregation,Finds the minimum non-null value of a numeric ...
4,trend,aggregation,Calculates the slope of the linear trend of va...
5,mean,aggregation,Computes the average value of a numeric feature.
6,avg_time_between,aggregation,Computes the average time between consecutive ...
7,skew,aggregation,Computes the skewness of a data set.
8,time_since_last,aggregation,Time since last related instance.
9,percent_true,aggregation,Finds the percent of 'True' values in a boolea...


In [68]:
es['transactions'].df.head(2)

,arrive_dt,transaction_id,session_id,transaction_time,amount,customer_id,device,session_start,zip_code,join_date,depart_dt,date_hour,product_id
2014-01-01 00:00:00,2014-01-01 00:00:00,352,1,2014-01-01 00:00:00,7.39,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:00:00,2014-01-01,4
2014-01-01 00:01:05,2014-01-01 00:01:05,186,1,2014-01-01 00:01:05,147.23,1,desktop,2014-01-01,60091,2008-01-01,2014-01-01 02:01:05,2014-01-01,4


In [38]:
break

SyntaxError: 'break' outside loop (<ipython-input-38-6aaf1f276005>, line 1)

In [2]:
ED = pd.read_pickle('./../3_Data/SGH_ED_bk.pkl')

In [3]:
IP = pd.read_pickle('./../3_Data/SGH_ward_BK.pkl')

In [4]:
df = make_callender_columns(ED,'arrival','arr')

In [5]:
i = 2000

In [6]:
ED.head(2)

,pseudo_patient_number,attendance_number,age,gender,department,site_attended,arrival,arrival_mode,arrival_mode_desc,1st_time_triaged_[datetime],...,age_group,arr_hour,arr_dayofweek,arr_day,arr_month,arr_week,arr_dayofweek_name,arr_year,arr_date,arr_flag_wkend
0,4101045105,90902325901,94.0,F,Main ED,Main ED,2009-04-13 09:16:00,NaN,BROUGHT IN BY AMBULANCE,NaT,...,75+,9,0,13,4,16,Monday,2009,2009-04-13,0
1,644868441371,90902326101,47.0,M,Main ED,Main ED,2009-04-13 09:22:00,NaN,BROUGHT IN BY AMBULANCE,NaT,...,36-64,9,0,13,4,16,Monday,2009,2009-04-13,0


In [7]:
ED.isnull().sum()

pseudo_patient_number                 1
attendance_number                     0
age                                  10
gender                               10
department                            0
site_attended                         0
arrival                               0
arrival_mode                     269946
arrival_mode_desc                     0
1st_time_triaged_[datetime]      280630
1st_time_seen_by_doctor           76978
time_of_inpatient_bed_request    658458
time_of_speciality_referral      653702
speciality_referred_to           653703
time_into_cdu                    816058
time_leaving_dept_inc_cdu             0
admission_flag                        0
majorsminorsresus_patient        114766
triage_category                  639560
age_group                            10
arr_hour                              0
arr_dayofweek                         0
arr_day                               0
arr_month                             0
arr_week                              0


# clean ED

In [8]:
ED.arrival_mode_desc.value_counts()

OTHER                      566738
BROUGHT IN BY AMBULANCE    347099
Name: arrival_mode_desc, dtype: int64

In [9]:
ED['arrival_mode_desc_bool'] = ED.arrival_mode_desc == 'BROUGHT IN BY AMBULANCE'

In [10]:
ED['arrival_mode_desc_bool'].value_counts()

False    566738
True     347099
Name: arrival_mode_desc_bool, dtype: int64

In [11]:
ED['admission_flag_bool'] = ED.admission_flag == 1

In [12]:
ED['admission_flag_bool'].value_counts()

False    559945
True     353892
Name: admission_flag_bool, dtype: int64

# create es

In [13]:
es = ft.EntitySet(id='activity')

In [14]:
es

Entityset: activity
  Entities:
  Relationships:
    No relationships

In [15]:
ED.arrival.dtypes

dtype('<M8[ns]')

In [16]:
df_ed = ED[['pseudo_patient_number','age','gender',
                    'attendance_number','arrival_mode_desc_bool',
                    'arrival','arr_date',
            'admission_flag'
                   ]]

for i in ED.columns:
    print(i)
    print(ED[i].dtype)

In [17]:
df_ed.head()

,pseudo_patient_number,age,gender,attendance_number,arrival_mode_desc_bool,arrival,arr_date,admission_flag
0,4101045105,94.0,F,90902325901,True,2009-04-13 09:16:00,2009-04-13,0
1,644868441371,47.0,M,90902326101,True,2009-04-13 09:22:00,2009-04-13,1
2,668666687357,0.0,M,90902326501,False,2009-04-13 09:31:00,2009-04-13,1
3,446747461391,4.0,M,90902328601,False,2009-04-13 10:52:00,2009-04-13,0
4,423646238168,31.0,F,90902328801,True,2009-04-13 11:18:00,2009-04-13,1


In [18]:
df_ed.tail()

,pseudo_patient_number,age,gender,attendance_number,arrival_mode_desc_bool,arrival,arr_date,admission_flag
1124659,497545978018,69.0,F,91803566401,True,2018-05-04 12:00:00,2018-05-04,0
1124671,812787124394,73.0,M,91804244501,True,2018-05-26 11:31:00,2018-05-26,1
1124673,073505730740,2.0,F,91803579801,False,2018-05-04 19:37:00,2018-05-04,0
1124681,757373573113,93.0,M,91804272201,True,2018-05-27 06:17:00,2018-05-27,1
1124686,854787547747,37.0,M,91804267301,True,2018-05-27 00:13:00,2018-05-27,0


In [19]:
df_ed.loc[0].arrival

Timestamp('2009-04-13 09:16:00')

In [20]:
df_ed.arrival.dtype

dtype('<M8[ns]')

In [21]:
ft.variable_types.Numeric

featuretools.variable_types.variable.Numeric

In [22]:
es = es.entity_from_dataframe(entity_id='EDatt',
                        dataframe=df_ed,
                        index='attendance_number',
                        time_index='arrival',
                             variable_types={'arrival':ft.variable_types.Datetime,
                                            'arr_date':ft.variable_types.Datetime,
                                             'pseudo_patient_number':ft.variable_types.Numeric,
                                             'age':ft.variable_types.Numeric,
                                             'attendance_number':ft.variable_types.Numeric
})

In [23]:
es

Entityset: activity
  Entities:
    EDatt [Rows: 913837, Columns: 8]
  Relationships:
    No relationships

In [24]:
es['EDatt']

Entity: EDatt
  Variables:
    attendance_number (dtype: index)
    gender (dtype: categorical)
    arrival_mode_desc_bool (dtype: boolean)
    admission_flag (dtype: numeric)
    arrival (dtype: datetime_time_index)
    arr_date (dtype: datetime)
    pseudo_patient_number (dtype: numeric)
    age (dtype: numeric)
  Shape:
    (Rows: 913837, Columns: 8)

#### make patient table

In [25]:
es = es.normalize_entity(base_entity_id='EDatt',
                   new_entity_id ='patients',
                   index='pseudo_patient_number',
                         make_time_index=True,
                   additional_variables=['gender'])

In [26]:
es['patients']

Entity: patients
  Variables:
    pseudo_patient_number (dtype: index)
    gender (dtype: categorical)
    first_EDatt_time (dtype: datetime_time_index)
  Shape:
    (Rows: 399661, Columns: 3)

#### create a day table - by normalising a new feature

In [27]:
es = es.normalize_entity(base_entity_id='EDatt',
                   new_entity_id="days",
                    index="arr_date")

#### import hourly table

In [28]:
hrly = pd.read_pickle('./../3_Data/processed/sgh/sghHOURLY.pkl')

In [29]:
hrly.columns

Index(['EDocc_nonadmit', 'EDocc_admit', 'EDocc_total', 'EDocc_awaitingadm',
       'EDocc_nonbreach', 'EDocc_breach', 'IPocc_NonElective',
       'IPocc_Day_Case', 'IPocc_Elective', 'IPocc_total', 'IPocc_elec_nonelec',
       'IPadm', 'IPadm_nonelec', 'IPadm_daycase', 'IPadm_elective', 'IPdis',
       'IPdis_nonelec', 'IPdis_daycase', 'IPdis_elective',
       'IPadm_elec_nonelec', 'IPdis_elec_nonelec', 'EDarrive',
       'EDarrive_breach', 'EDarrive_adm', 'EDdepart', 'EDdepart_breach',
       'EDdepart_adm', 'datetime', 'dt_hour', 'dt_dayofweek', 'dt_day',
       'dt_month', 'dt_week', 'dt_dayofweek_name', 'dt_year', 'dt_date',
       'dt_flag_wkend'],
      dtype='object')

In [30]:
hrly2 = hrly[['dt_date','EDocc_nonadmit', 'EDocc_admit', 'EDocc_total', 'EDocc_awaitingadm',
       'EDocc_nonbreach', 'EDocc_breach', 'IPocc_NonElective',
       'IPocc_Day_Case', 'IPocc_Elective', 'IPocc_total', 'IPocc_elec_nonelec',
       'IPadm', 'IPadm_nonelec', 'IPadm_daycase', 'IPadm_elective', 'IPdis',
       'IPdis_nonelec', 'IPdis_daycase', 'IPdis_elective',
       'IPadm_elec_nonelec', 'IPdis_elec_nonelec', 'EDarrive',
       'EDarrive_breach', 'EDarrive_adm', 'EDdepart', 'EDdepart_breach',
       'EDdepart_adm']].reset_index()

In [31]:
es.entity_from_dataframe(entity_id='hourly',dataframe=hrly2,index='index')

Entityset: activity
  Entities:
    EDatt [Rows: 913837, Columns: 7]
    patients [Rows: 399661, Columns: 3]
    days [Rows: 3470, Columns: 2]
    hourly [Rows: 83295, Columns: 29]
  Relationships:
    EDatt.pseudo_patient_number -> patients.pseudo_patient_number
    EDatt.arr_date -> days.arr_date

In [32]:
new_relationship = ft.Relationship(es["days"]["arr_date"],
                                    es["hourly"]["dt_date"])

es = es.add_relationship(new_relationship)

In [33]:
es['hourly'].df.head(2)

,index,dt_date,EDocc_nonadmit,EDocc_admit,EDocc_total,EDocc_awaitingadm,EDocc_nonbreach,EDocc_breach,IPocc_NonElective,IPocc_Day_Case,...,IPdis_daycase,IPdis_elective,IPadm_elec_nonelec,IPdis_elec_nonelec,EDarrive,EDarrive_breach,EDarrive_adm,EDdepart,EDdepart_breach,EDdepart_adm
2009-04-01 00:00:00,2009-04-01 00:00:00,2009-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,3.0,4.0,0.0,0.0,0.0
2009-04-01 01:00:00,2009-04-01 01:00:00,2009-04-01,2.0,4.0,6.0,0.0,3.0,3.0,3.0,2.0,...,0.0,0.0,0.0,0.0,6.0,3.0,4.0,1.0,0.0,0.0


# summary of esets

In [34]:
es

Entityset: activity
  Entities:
    EDatt [Rows: 913837, Columns: 7]
    patients [Rows: 399661, Columns: 3]
    days [Rows: 3470, Columns: 2]
    hourly [Rows: 83295, Columns: 29]
  Relationships:
    EDatt.pseudo_patient_number -> patients.pseudo_patient_number
    EDatt.arr_date -> days.arr_date
    hourly.dt_date -> days.arr_date

In [35]:
es['days'].df.head()

,arr_date,first_EDatt_time
2009-04-01,2009-04-01,2009-04-01 00:04:00
2009-04-02,2009-04-02,2009-04-02 00:05:00
2009-04-03,2009-04-03,2009-04-03 00:01:00
2009-04-04,2009-04-04,2009-04-04 00:04:00
2009-04-05,2009-04-05,2009-04-05 00:03:00


In [36]:
es['EDatt'].df.head(2)

,attendance_number,arrival_mode_desc_bool,admission_flag,arrival,arr_date,pseudo_patient_number,age
90902041001,90902041001,False,0,2009-04-01 00:04:00,2009-04-01,4.930409e+11,19.0
90902041101,90902041101,True,0,2009-04-01 00:19:00,2009-04-01,6.104641e+11,59.0


In [37]:
es['patients'].df.head(3)

,pseudo_patient_number,gender,first_EDatt_time
4.930409e+11,4.930409e+11,F,2009-04-01 00:04:00
6.104641e+11,6.104641e+11,F,2009-04-01 00:19:00
7.723737e+11,7.723737e+11,M,2009-04-01 00:19:00


In [38]:
es['hourly'].df.head(2)

,index,dt_date,EDocc_nonadmit,EDocc_admit,EDocc_total,EDocc_awaitingadm,EDocc_nonbreach,EDocc_breach,IPocc_NonElective,IPocc_Day_Case,...,IPdis_daycase,IPdis_elective,IPadm_elec_nonelec,IPdis_elec_nonelec,EDarrive,EDarrive_breach,EDarrive_adm,EDdepart,EDdepart_breach,EDdepart_adm
2009-04-01 00:00:00,2009-04-01 00:00:00,2009-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,3.0,4.0,0.0,0.0,0.0
2009-04-01 01:00:00,2009-04-01 01:00:00,2009-04-01,2.0,4.0,6.0,0.0,3.0,3.0,3.0,2.0,...,0.0,0.0,0.0,0.0,6.0,3.0,4.0,1.0,0.0,0.0


In [39]:
es['days'].df.head(3)

,arr_date,first_EDatt_time
2009-04-01,2009-04-01,2009-04-01 00:04:00
2009-04-02,2009-04-02,2009-04-02 00:05:00
2009-04-03,2009-04-03,2009-04-03 00:01:00


# make features

feature_matrix, feature_defs = ft.dfs(entityset=es,target_entity='days',
      agg_primitives=['count','mean','num_true','sum'],
      trans_primitives=['month','year','day'],
    max_depth=1,n_jobs=2)

feature_matrix

In [63]:
feature_matrix1, feature_defs = ft.dfs(entityset=es,target_entity='days',
      agg_primitives=['count','mean','num_true','sum','max','last'],
      trans_primitives=['month','year','weekend','weekday'],
    max_depth=1,n_jobs=1)

feature_matrix1.shape

(3470, 125)

In [64]:
feature_matrix2, feature_defs = ft.dfs(entityset=es,target_entity='days',
      agg_primitives=['count','mean','num_true','sum','max','last'],
      trans_primitives=['month','year','weekend','weekday'],
    max_depth=2,n_jobs=1)

feature_matrix2.shape

(3470, 135)

In [65]:
feature_matrix, feature_defs = ft.dfs(entityset=es,target_entity='days',
      agg_primitives=['count','mean','num_true','sum','max','last'],
      trans_primitives=['month','year','weekend','weekday'],
    max_depth=3,n_jobs=1)

feature_matrix.shape

(3470, 179)

In [66]:
set(feature_matrix.columns) - set(feature_matrix1.columns)

{'LAST(EDatt.MONTH(arrival))',
 'LAST(EDatt.WEEKDAY(arrival))',
 'LAST(EDatt.WEEKEND(arrival))',
 'LAST(EDatt.YEAR(arrival))',
 'LAST(EDatt.patients.COUNT(EDatt))',
 'LAST(EDatt.patients.LAST(EDatt.admission_flag))',
 'LAST(EDatt.patients.LAST(EDatt.age))',
 'LAST(EDatt.patients.LAST(EDatt.arr_date))',
 'LAST(EDatt.patients.MAX(EDatt.admission_flag))',
 'LAST(EDatt.patients.MAX(EDatt.age))',
 'LAST(EDatt.patients.MEAN(EDatt.admission_flag))',
 'LAST(EDatt.patients.MEAN(EDatt.age))',
 'LAST(EDatt.patients.MONTH(first_EDatt_time))',
 'LAST(EDatt.patients.NUM_TRUE(EDatt.arrival_mode_desc_bool))',
 'LAST(EDatt.patients.SUM(EDatt.admission_flag))',
 'LAST(EDatt.patients.SUM(EDatt.age))',
 'LAST(EDatt.patients.WEEKDAY(first_EDatt_time))',
 'LAST(EDatt.patients.YEAR(first_EDatt_time))',
 'LAST(EDatt.patients.gender)',
 'MAX(EDatt.patients.COUNT(EDatt))',
 'MAX(EDatt.patients.LAST(EDatt.admission_flag))',
 'MAX(EDatt.patients.LAST(EDatt.age))',
 'MAX(EDatt.patients.MAX(EDatt.admission_flag))',

In [41]:
for i in feature_matrix.columns:
    print(i)

COUNT(EDatt)
MEAN(EDatt.admission_flag)
MEAN(EDatt.age)
NUM_TRUE(EDatt.arrival_mode_desc_bool)
SUM(EDatt.admission_flag)
SUM(EDatt.age)
MAX(EDatt.admission_flag)
MAX(EDatt.age)
LAST(EDatt.arrival_mode_desc_bool)
LAST(EDatt.admission_flag)
LAST(EDatt.pseudo_patient_number)
LAST(EDatt.age)
COUNT(hourly)
MEAN(hourly.EDocc_nonadmit)
MEAN(hourly.EDocc_admit)
MEAN(hourly.EDocc_total)
MEAN(hourly.EDocc_awaitingadm)
MEAN(hourly.EDocc_nonbreach)
MEAN(hourly.EDocc_breach)
MEAN(hourly.IPocc_NonElective)
MEAN(hourly.IPocc_Day_Case)
MEAN(hourly.IPocc_Elective)
MEAN(hourly.IPocc_total)
MEAN(hourly.IPocc_elec_nonelec)
MEAN(hourly.IPadm)
MEAN(hourly.IPadm_nonelec)
MEAN(hourly.IPadm_daycase)
MEAN(hourly.IPadm_elective)
MEAN(hourly.IPdis)
MEAN(hourly.IPdis_nonelec)
MEAN(hourly.IPdis_daycase)
MEAN(hourly.IPdis_elective)
MEAN(hourly.IPadm_elec_nonelec)
MEAN(hourly.IPdis_elec_nonelec)
MEAN(hourly.EDarrive)
MEAN(hourly.EDarrive_breach)
MEAN(hourly.EDarrive_adm)
MEAN(hourly.EDdepart)
MEAN(hourly.EDdepart_bre

In [42]:
feature_matrix.shape

(3470, 179)

In [43]:
feature_matrix.shape

(3470, 179)

## do some basic drops

In [44]:
uniques = feature_matrix.apply(lambda x: x.nunique())

In [45]:
uniques[uniques==1].index

Index(['MAX(EDatt.admission_flag)', 'COUNT(hourly)',
       'MAX(EDatt.patients.MEAN(EDatt.admission_flag))',
       'MAX(EDatt.patients.MAX(EDatt.admission_flag))',
       'MAX(EDatt.patients.LAST(EDatt.admission_flag))'],
      dtype='object')

In [46]:
feature_matrix = feature_matrix.drop(uniques[uniques==1].index, axis=1)

#### drop columns which have all same values

In [47]:
feature_matrix.shape

(3470, 174)

In [48]:
cols = list(feature_matrix)
nunique = feature_matrix.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
feature_matrix.drop(cols_to_drop, axis=1,inplace=True)

In [49]:
feature_matrix.shape

(3470, 174)

#### drop cols that are duplicated 

In [50]:
from utils import duplicate_columns

In [51]:
dups = duplicate_columns(feature_matrix)
dups

['WEEKEND(first_EDatt_time)',
 'LAST(EDatt.WEEKEND(arrival))',
 'MONTH(first_EDatt_time)',
 'YEAR(first_EDatt_time)',
 'WEEKDAY(first_EDatt_time)',
 'LAST(EDatt.MONTH(arrival))',
 'LAST(EDatt.YEAR(arrival))',
 'LAST(EDatt.WEEKDAY(arrival))',
 'MEAN(hourly.IPadm_daycase)',
 'SUM(hourly.IPadm_daycase)',
 'MEAN(EDatt.patients.MAX(EDatt.age))',
 'SUM(EDatt.patients.MAX(EDatt.age))',
 'MAX(EDatt.patients.MAX(EDatt.age))',
 'LAST(EDatt.patients.MAX(EDatt.age))']

In [52]:
feature_matrix.drop(dups,axis=1,inplace=True)

In [53]:
feature_matrix.shape

(3470, 160)

### save out

In [54]:
pd.to_pickle(feature_matrix,'./activity.pkl')

In [55]:
ft.list_primitives().head(40)

,name,type,description
0,all,aggregation,Test if all values are 'True'.
1,num_unique,aggregation,Returns the number of unique categorical varia...
2,last,aggregation,Returns the last value.
3,percent_true,aggregation,Finds the percent of 'True' values in a boolea...
4,median,aggregation,Finds the median value of any feature with wel...
5,mean,aggregation,Computes the average value of a numeric feature.
6,mode,aggregation,Finds the most common element in a categorical...
7,std,aggregation,Finds the standard deviation of a numeric feat...
8,trend,aggregation,Calculates the slope of the linear trend of va...
9,count,aggregation,Counts the number of non null values.


In [56]:
ft.list_primitives().tail(40).loc[22].description

'Transform Datetime feature into the boolean of Weekend.'

# Dev

In [57]:
break

SyntaxError: 'break' outside loop (<ipython-input-57-6aaf1f276005>, line 1)

In [ ]:
ED.columns

#### create and link day table - by adding a new df

from core import create_timeseries_from_events

ED.columns

start = ED.tail(i).arrival.min().round('D')
end = ED.tail(i).arrival.max().round('D')

occED = create_timeseries_from_events(ED.tail(i),'arrival','time_leaving_dept_inc_cdu',col_to_split='admission_flag',start=start,end=end,freq='H')

occED['EDocc_MEAN'] = occED.sum(axis=1) # make agg col

occED['day'] = occED.index.round('D')

occEDday = occED.groupby(['day']).mean()

occEDday.index

day = occEDday['EDocc_MEAN'].reset_index()

day.head(2)

#### add to es

es = es.entity_from_dataframe(entity_id="days",dataframe=day,
                              index="day",
                              time_index="day")

new_relationship = ft.Relationship(es["days"]["day"],
                                   es["ED_atten"]["arr_date"])

es = es.add_relationship(new_relationship)

#### Look at tables

In [ ]:
es['days'].df.head()

es['days'].df['day'].head()

es['days'].df['day'].dtype

es['ED_atten'].index

es['ED_atten'].df.arrival.dtype

es['patient'].df.columns #.dtype

es['patient'].df['first_ED_atten_time'].dtype

In [ ]:
es['ED_atten']

In [ ]:
es['patient']

In [ ]:
es['days']

es['days']

In [ ]:
es

# DFS

### create range of datetimes for arrival in df

In [ ]:
feature_matrix, feature_defs = ft.dfs(entityset=es,target_entity="days")

In [ ]:
feature_matrix

In [ ]:
window_fm, window_features = ft.dfs(entityset=es,target_entity='days',cutoff_time=ct,
       cutoff_time_in_index=True,
      training_window = '24 hours')

In [ ]:
pd.DatetimeIndex(start=start,end=end,freq='D')

In [ ]:
ft.list_primitives().head(40)